In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

## Training data

In [2]:
df_2016 = pd.read_csv("../Daten/kelmarsh_scada_2016/kelmarsh_turbine1.csv")
df_2017 = pd.read_csv("../Daten/kelmarsh_scada_2017/kelmarsh_turbine1.csv")
df_2018 = pd.read_csv("../Daten/kelmarsh_scada_2018/kelmarsh_turbine1.csv")
df_2019 = pd.read_csv("../Daten/kelmarsh_scada_2019/kelmarsh_turbine1.csv")

In [ ]:
dfs = [df_2016, df_2017, df_2018, df_2019]

df_combined = pd.concat(dfs, ignore_index=True)

In [ ]:
cutoff_date = "2019-06-30 23:50:00"
df_train = df_combined[df_combined["Date and time"] <= cutoff_date]

In [ ]:
df_train["Wind speed, Maximum (m/s)"]

In [ ]:
df_train['Date and time'] = pd.to_datetime(df_train['Date and time'], format='mixed', dayfirst=True, errors='coerce')

In [ ]:
import matplotlib.pyplot as plt

df_train['Month'] = df_train['Date and time'].dt.month

df_train.boxplot(column='Wind speed (m/s)', by='Month', grid=False, showfliers=True, figsize=(10,6), notch=True)

plt.title("Monatliche Verteilung der Windgeschwindigkeit")
plt.suptitle("")
plt.xlabel("Monat")
plt.ylabel("Windgeschwindigkeit (m/s)")
plt.xticks(rotation=0)
plt.rcParams.update({'font.size': 14})
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df_train['Hour'] = df_train['Date and time'].dt.hour

df_train.boxplot(column='Wind speed (m/s)', by='Hour', grid=False, showfliers=True, figsize=(10,6), notch=True)

plt.title("Stündliche Verteilung der Windgeschwindigkeit")
plt.suptitle("")
plt.xlabel("Stunde")
plt.ylabel("Windgeschwindigkeit (m/s)")
plt.xticks(rotation=0)
plt.rcParams.update({'font.size': 14})
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

filtered_df = df_train['Wind speed (m/s)']

bin_edges = np.arange(0, filtered_df.max() + 1, 1)

plt.figure(figsize=(10, 6))
plt.hist(filtered_df, bins=bin_edges, density=False, alpha=0.6, color='skyblue', edgecolor='black')
plt.title("Histogram der Windgeschwindigkeiten (Kelmarsh)")
plt.xlabel("Windgeschwindidkeit (m/s)")
plt.ylabel("Frequenz")
plt.grid(True)

plt.rcParams.update({'font.size': 19})
plt.show()

In [1]:
plt.figure(figsize=(10, 6))
plt.scatter(df_train["Density adjusted wind speed (m/s)"], df_train['Power (kW)'])
plt.xlabel('Wind Geschwindigkeit (m/s)')
plt.ylabel('Leistung (kW)')
plt.title('Leistungskurve vor Bereinigung der Daten')
plt.grid(True)

plt.rcParams.update({'font.size': 18})
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=(14, 6))
plt.scatter(df_train['Date and time'], df_train["Lost Production (Time-based IEC B.3.2) (kWh)"])
plt.xlabel('Date')
plt.ylabel('Lost Production (kWh)')
plt.title('Production Loss')
plt.grid(True)
plt.show()

In [ ]:

numerische_spalten = df_train.select_dtypes(include=['float64', 'int64'])

korrelationen = numerische_spalten.corr()['Power (kW)']
korrelationen = korrelationen.dropna()


In [ ]:
korrelationen = korrelationen.sort_values()
print(korrelationen)

In [ ]:
threshold = 0

condition = ((df_train["Lost Production Total (kWh)"] >= threshold) | (df_train["Lost Production Total (kWh)"] <= -20)) & (df_train["Wind speed (m/s)"] >= 2.5)

df_train = df_train[~condition]

print(df_train)

In [ ]:

condition = (df_train["Yaw bearing angle (°)"] != df_train["Wind direction (°)"]) & (df_train["Wind speed (m/s)"] > 11) & (df_train["Power (kW)"] < 1800)
cond = df_train[condition]
print(cond[["Power (kW)", "Wind speed (m/s)"]])

df_train = df_train[~condition]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_train["Density adjusted wind speed (m/s)"], df_train['Power (kW)'])
plt.xlabel('Wind Geschwindigkeit (m/s)')
plt.ylabel('Leistung (kW)')
plt.title('Leistungskurve nach Bereinigung der Daten')
plt.grid(True)

plt.rcParams.update({'font.size': 18})
plt.show()

In [ ]:
relevant_columns = [
    "Date and time",
    "Density adjusted wind speed (m/s)", "Wind direction (°)", "Nacelle position (°)",
    "Power (kW)", "Wind speed (m/s)"
]
df_train = df_train[relevant_columns]

In [ ]:
df_train.describe()

In [ ]:
df_train["Power (kW)"][df_train["Power (kW)"] < 0] = 0

In [ ]:
df_train.describe()

In [ ]:
df_train.isna().sum()

In [ ]:
df_numeric = df_train.select_dtypes(include=["float64", "int64"])
df_train[df_numeric.columns] = df_numeric.interpolate().fillna(method="bfill").fillna(method="ffill")

In [ ]:
df_train.isna().sum()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_one_day = df_train.iloc[-144:]

plt.figure(figsize=(12, 6))
plt.plot(df_one_day.index, df_one_day['Power (kW)'], label='Power (kW)', marker='o')
plt.plot(df_one_day.index, df_one_day['Wind speed (m/s)'], label='Wind speed (m/s)', marker='x')
plt.title('Power (kW) and Wind Speed (m/s) for One Day')
plt.xlabel('Time (Index)')
plt.ylabel('Values')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

df_one_day[['Power (kW)', 'Wind speed (m/s)']].to_csv('beispiel.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

start = 10000
end = start + 144 * 60

df = pd.DataFrame({'power': df_train["Wind speed (m/s)"].iloc[0:end]})

print(df['power'])
df['MA_24h'] = df['power'].rolling(window=144).mean()  # 24-Stunden gleitender Durchschnitt
df['MA_7d'] = df['power'].rolling(window=144*7).mean()  # 7 gleitender Durchschnitt

plt.figure(figsize=(12, 6))
plt.plot(df_train["Date and time"].iloc[start:end], df["power"].iloc[start:end], label="Rohdaten", alpha=0.5)
plt.plot(df_train["Date and time"].iloc[start:end], df['MA_24h'].iloc[start:end], label="Gleitender Durchschnitt (24 Stunden)", linewidth=2)
plt.plot(df_train["Date and time"].iloc[start:end], df['MA_7d'].iloc[start:end], label="Gleitender Durchschnitt (7 Tage)", linewidth=2)
plt.title("Windgeschwindigkeit mit gleitendem Durchschnitt")
plt.xlabel("Datum")
plt.ylabel("Windgeschwindigkeit (m/s)")
plt.legend()
import matplotlib.dates as mdates

plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=288))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.rcParams.update({'font.size': 16})
plt.grid()
plt.show()

In [ ]:
fft_result = np.fft.fft(df['power'])
frequencies = np.fft.fftfreq(len(df['power']), d=(144))

amplitude = np.abs(fft_result)

# Plot des Spektrums
plt.figure(figsize=(10, 5))
plt.plot(frequencies[:len(frequencies)//2], amplitude[:len(frequencies)//2])
plt.title('Fourier-Analyse - Frequenzspektrum')
plt.xlabel('Frequenz (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

In [ ]:
output_file = "../Daten/train_data.csv"
df_train.to_csv(output_file, index=False)

print(f"Gefilterte Daten wurden als '{output_file}' gespeichert.")

In [ ]:
COLUMNS = [
    "Density adjusted wind speed (m/s)", "Wind direction (°)", "Nacelle position (°)",
    "Power (kW)", "Wind speed (m/s)"
]

## Validation data

In [ ]:
df_2020 = pd.read_csv("../Daten/kelmarsh_scada_2020/kelmarsh_turbine1.csv")

In [ ]:
dfs = [df_2019, df_2020]

df_combined_val = pd.concat(dfs, ignore_index=True)

In [ ]:
relevant_columns = [
    "Date and time",
    "Density adjusted wind speed (m/s)", "Wind direction (°)", "Nacelle position (°)",
    "Power (kW)", "Wind speed (m/s)"
]
df_combined_val = df_combined_val[relevant_columns]

In [ ]:
cutoff_date = "2020-06-30 23:50:00"
start_date = "2019-07-01 00:00:00"
df_val = df_combined_val[df_combined_val["Date and time"] >= start_date]

df_val = df_val[df_val["Date and time"] <= cutoff_date]

In [ ]:
df_val.last

In [ ]:
df_val["Power (kW)"][df_val["Power (kW)"] < 0] = 0

In [ ]:
df_numeric = df_val.select_dtypes(include=["float64", "int64"])
df_val[df_numeric.columns] = df_numeric.interpolate().fillna(method="bfill").fillna(method="ffill")

In [ ]:
output_file = "../Daten/validation_data.csv"
df_val.to_csv(output_file, index=False)

print(f"Gefilterte Daten wurden als '{output_file}' gespeichert.")

## Test data

In [ ]:
df_2021 = pd.read_csv("../Daten/kelmarsh_scada_2021/kelmarsh_turbine1.csv")

In [ ]:
dfs = [df_2020, df_2021]

df_combined_val = pd.concat(dfs, ignore_index=True)

In [ ]:
relevant_columns = [
    "Date and time",
    "Wind speed (m/s)", "Wind direction (°)", "Nacelle position (°)",
    "Power (kW)", "Density adjusted wind speed (m/s)"
]
df_combined_val = df_combined_val[relevant_columns]

In [ ]:
start_date = "2020-07-01 00:00:00"
df_test = df_combined_val[df_combined_val["Date and time"] >= start_date]

In [ ]:
df_test.last

In [ ]:
df_test["Power (kW)"][df_test["Power (kW)"] < 0] = 0

In [ ]:
duplicates = df_test.columns[df_test.columns.duplicated()]
print("Doppelte Spaltennamen:", duplicates)

In [ ]:
df_test = df_test.loc[:, ~df_test.columns.duplicated()]

In [ ]:
df_numeric = df_test.select_dtypes(include=["float64", "int64"])
df_test.loc[:, df_numeric.columns] = df_numeric.interpolate().fillna(method="bfill").fillna(method="ffill")

In [ ]:
output_file = "../Daten/test_data.csv"
df_test.to_csv(output_file, index=False)

print(f"Gefilterte Daten wurden als '{output_file}' gespeichert.")